In [1]:
import tensorflow as tf

In [2]:
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


# create some helper functions

In [4]:
# Init weights

In [5]:
def init_weights(shape):
    init_random_dist = tf.truncated_normal(shape,stddev=0.1) # keeps mean around zero and .1 makes it narrow
    return tf.Variable(init_random_dist)

In [6]:
# init bias

In [7]:
def init_bias(shape):
    # have every bias be same value
    init_bias_vals = tf.constant(0.1, shape=shape)
    # now return in a TF var so we don't need to do it later
    return tf.Variable(init_bias_vals)

In [8]:
# return 2d convolution

In [9]:
def conv2d(x,W):
    # x is our actual input tensor: [batch, H, W, Channels]
    # W is our kernel: [filter H, filter W, channels IN, Channels OUT]
    
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],  padding='SAME') # padding 2 codes SAME or VALID

In [10]:
# pooling helper function

In [11]:
def max_pool_2by2(x):
    # x --> [batch, h, w, c] BHWC (called NHWC in docs)
    # ksize will add up B & C, but we don't want to touch them so keep them as 1
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME') 

# Now create layers

In [12]:
# CONVOLUTIONAL LAYER
def convolutional_layer(input_x, shape):
    W = init_weights(shape)
    b = init_bias([shape[3]])
    # RELU --> rectified linear unit
    return tf.nn.relu(conv2d(input_x,W)+b)

In [13]:
# normal (fully connected) layer
def normal_full_layer(input_layer, size):
    input_size = int(input_layer.get_shape()[1])# .getshape()[1])   ## should be get_shape, error is thrown the first time we 
    W = init_weights([input_size, size])
    b = init_bias([size])
    return tf.matmul(input_layer, W) + b

In [14]:
# placeholders
x = tf.placeholder(tf.float32, shape=[None, 784]) # 784 is pixels in data

In [15]:
y_true = tf.placeholder(tf.float32, shape=[None,10]) # because images are one-hot encoded, they are encoded in 0-9

## create the layers

In [16]:
# image layer == input
x_image = tf.reshape(x,[-1,28,28,1]) # original image array is flat, so we need to make 728 pixels into an image again (sqrt(784) = 28)

In [17]:
convolution_1 = convolutional_layer(x_image, shape=[5,5,1,32]) 
# 5x5 is the convolutional layer, 5x5 is patch size. 
# Next is input channels, which is 1 because grayscale, 32 is "actual features we're computing" "Number of output channels"
convolution_1_pooling = max_pool_2by2(convolution_1)

In [18]:
convolution_2 = convolutional_layer(convolution_1_pooling, shape=[5,5,32,64]) # 5x5 patch, now 32 features in, want 64 features per patch
convolution_2_pooling = max_pool_2by2(convolution_2)

In [19]:
# flatten out so we can pass it to a fully connected layer
convo_2_flat = tf.reshape(convolution_2_pooling, [-1, 7*7*64])
full_layer_one = tf.nn.relu(normal_full_layer(convo_2_flat, 1024))
## here is where we get the get_shape error!!

In [20]:
# DROPOUT
hold_prob = tf.placeholder(tf.float32)
full_one_dropout = tf.nn.dropout(full_layer_one,keep_prob=hold_prob)

In [21]:
y_pred = normal_full_layer(full_one_dropout, 10) # 10 different glyphs 0-9, it determines the prob that it's each glyph